![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import time
import warnings
warnings.filterwarnings("ignore")

filename_features = "./Dados/UCI HAR Dataset/features.txt"
filename_labels = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./Dados/UCI HAR Dataset/test/y_test.txt"
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")['nome_var']
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])['subject_id']
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])['subject_id']
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [20]:
%%time

def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean())/s.std()
    return s

X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
display(X_train_pad.head())
X_test_pad = pd.DataFrame(X_test).apply(padroniza, axis=0)
display(X_test_pad.head())

,1 tBodyAcc-mean()-X,2 tBodyAcc-mean()-Y,3 tBodyAcc-mean()-Z,4 tBodyAcc-std()-X,5 tBodyAcc-std()-Y,6 tBodyAcc-std()-Z,7 tBodyAcc-mad()-X,8 tBodyAcc-mad()-Y,9 tBodyAcc-mad()-Z,10 tBodyAcc-max()-X,...,552 fBodyBodyGyroJerkMag-meanFreq(),553 fBodyBodyGyroJerkMag-skewness(),554 fBodyBodyGyroJerkMag-kurtosis(),"555 angle(tBodyAccMean,gravity)","556 angle(tBodyAccJerkMean),gravityMean)","557 angle(tBodyGyroMean,gravityMean)","558 angle(tBodyGyroJerkMean,gravityMean)","559 angle(X,gravityMean)","560 angle(Y,gravityMean)","561 angle(Z,gravityMean)"
0,0.200628,-0.063678,-0.419600,-0.868755,-0.939377,-0.737479,-0.859758,-0.938955,-0.766385,-0.855978,...,-0.795305,0.025958,-0.276380,-0.360579,0.062935,-0.778374,-0.026079,-0.687172,0.407918,-0.007567
1,0.055944,0.031484,-0.253891,-0.875366,-0.923839,-0.849247,-0.868472,-0.921936,-0.848870,-0.871300,...,0.130605,-0.897296,-0.767938,0.133002,-0.021460,-1.218722,1.484369,-0.694091,0.409089,0.007875
2,0.073510,-0.043414,-0.076289,-0.868980,-0.907698,-0.893724,-0.863078,-0.898793,-0.896640,-0.863264,...,1.152257,-0.260860,-0.438286,-0.377815,0.391949,0.151197,1.704085,-0.702191,0.410260,0.026501
3,0.066691,-0.208407,-0.249695,-0.870566,-0.939959,-0.921743,-0.864445,-0.938060,-0.925216,-0.863264,...,1.112694,0.591005,0.463123,-0.135016,-0.033635,1.037781,-1.002951,-0.701636,0.414622,0.031712
4,0.030467,0.027585,-0.109840,-0.875128,-0.934815,-0.921281,-0.867325,-0.931726,-0.927965,-0.870201,...,-0.149567,-0.138505,-0.240296,0.340383,0.268468,1.125841,-1.276196,-0.700104,0.425434,0.045222


,1 tBodyAcc-mean()-X,2 tBodyAcc-mean()-Y,3 tBodyAcc-mean()-Z,4 tBodyAcc-std()-X,5 tBodyAcc-std()-Y,6 tBodyAcc-std()-Z,7 tBodyAcc-mad()-X,8 tBodyAcc-mad()-Y,9 tBodyAcc-mad()-Z,10 tBodyAcc-max()-X,...,552 fBodyBodyGyroJerkMag-meanFreq(),553 fBodyBodyGyroJerkMag-skewness(),554 fBodyBodyGyroJerkMag-kurtosis(),"555 angle(tBodyAccMean,gravity)","556 angle(tBodyAccJerkMean),gravityMean)","557 angle(tBodyGyroMean,gravityMean)","558 angle(tBodyGyroJerkMean,gravityMean)","559 angle(X,gravityMean)","560 angle(Y,gravityMean)","561 angle(Z,gravityMean)"
0,-0.277661,-0.210595,2.192704,-0.787134,-0.833071,-0.093428,-0.807954,-0.838871,-0.104160,-0.824606,...,-0.253620,-0.166362,-0.344707,0.003565,0.357512,-1.363669,0.575389,-0.404722,0.622618,-0.038342
1,0.198626,0.182560,-0.250221,-0.876836,-0.928903,-0.857905,-0.896993,-0.928791,-0.863122,-0.824606,...,-2.300359,0.490938,0.012254,-0.264048,0.030782,-0.747105,1.870875,-0.361677,0.636623,-0.145684
2,0.024583,-0.318001,-0.228446,-0.921442,-0.933896,-0.906953,-0.916735,-0.933654,-0.912484,-0.910827,...,-0.291514,0.274775,-0.134966,-0.119651,0.445996,0.037913,0.323883,-0.370869,0.632738,-0.126834
3,-0.061047,-0.572940,-0.213673,-0.923681,-0.940657,-0.918926,-0.918996,-0.941439,-0.927619,-0.909586,...,-0.058615,-0.210642,-0.441639,-0.066432,0.338455,0.472614,0.625768,-0.363372,0.645167,-0.117565
4,0.013820,-0.387815,-0.494555,-0.921523,-0.928878,-0.949818,-0.915977,-0.923689,-0.951235,-0.909586,...,-1.090596,-0.810389,-0.796804,-0.022272,-0.098513,1.097162,-0.201965,-0.350197,0.665544,-0.104100


CPU times: total: 484 ms
Wall time: 493 ms


In [36]:
pca_sem_pad = PCA()
pca_sem_pad.fit(X_train)

df_sem_pad = pd.DataFrame({
'Variância Explicada': pca_sem_pad.explained_variance_,
'Variância Percentual': pca_sem_pad.explained_variance_ratio_,
'Variância Acumulada': np.cumsum(pca_sem_pad.explained_variance_),
'Variância Percentual Acumulada': np.cumsum(pca_sem_pad.explained_variance_ratio_)
})
df_sem_pad.index = df_sem_pad.index + 1 # Começar o índice do componente em 1

n_comp_sem_pad = np.where(df_sem_pad['Variância Percentual Acumulada'] >= 0.90)[0][0] + 1

print("--- Análise sem Padronização ---")
print(df_sem_pad.head())
print(f"\nComponentes para explicar 90% da variância: {n_comp_sem_pad}\n")

--- Análise sem Padronização ---
   Variância Explicada  Variância Percentual  Variância Acumulada  \
1            34.823630              0.625544            34.823630   
2             2.735046              0.049130            37.558677   
3             2.294393              0.041215            39.853070   
4             1.043775              0.018750            40.896845   
5             0.943517              0.016949            41.840362   

   Variância Percentual Acumulada  
1                        0.625544  
2                        0.674675  
3                        0.715889  
4                        0.734639  
5                        0.751587  

Componentes para explicar 90% da variância: 34



In [37]:
pca_com_pad = PCA()
pca_com_pad.fit(X_train_pad)

df_com_pad = pd.DataFrame({
'Variância Explicada': pca_com_pad.explained_variance_,
'Variância Percentual': pca_com_pad.explained_variance_ratio_,
'Variância Acumulada': np.cumsum(pca_com_pad.explained_variance_),
'Variância Percentual Acumulada': np.cumsum(pca_com_pad.explained_variance_ratio_)
})
df_com_pad.index = df_com_pad.index + 1 # Começar o índice do componente em 1

n_comp_com_pad = np.where(df_com_pad['Variância Percentual Acumulada'] >= 0.90)[0][0] + 1

print("--- Análise com Padronização ---")
print(df_com_pad.head())
print(f"\nComponentes para explicar 90% da variância: {n_comp_com_pad}\n")

--- Análise com Padronização ---
   Variância Explicada  Variância Percentual  Variância Acumulada  \
1           284.882377              0.507812           284.882377   
2            36.917616              0.065807           321.799993   
3            15.744110              0.028064           337.544103   
4            14.047175              0.025040           351.591278   
5            10.593279              0.018883           362.184557   

   Variância Percentual Acumulada  
1                        0.507812  
2                        0.573619  
3                        0.601683  
4                        0.626722  
5                        0.645605  

Componentes para explicar 90% da variância: 63



## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [38]:
%%time
n = 10 # Número de componentes principais
prcomp = PCA(n).fit(X_train_pad)

pc_treino = prcomp.transform(X_train_pad)
pc_teste  = prcomp.transform(X_test_pad)
pc_treino.shape

clf_pc = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001)
clf_pc.fit(pc_treino, y_train)
y_pred_pc = clf_pc.predict(pc_teste)
accuracy_train_pc = accuracy_score(y_train, clf_pc.predict(pc_treino))
accuracy_test_pc = accuracy_score(y_test, y_pred_pc)
print(f"Acurácia do modelo Decision Tree com PCA e padronização (treinamento): {accuracy_train_pc:.4f}")
print(f"Acurácia do modelo Decision Tree com PCA e padronização (teste): {accuracy_test_pc:.4f}")

Acurácia do modelo Decision Tree com PCA e padronização (treinamento): 0.8587
Acurácia do modelo Decision Tree com PCA e padronização (teste): 0.7737
CPU times: total: 531 ms
Wall time: 224 ms


In [39]:
%%time
n = 10 # Número de componentes principais
prcomp = PCA(n).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)
pc_treino.shape

clf_pc = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001)
clf_pc.fit(pc_treino, y_train)
y_pred_pc = clf_pc.predict(pc_teste)
accuracy_train_pc = accuracy_score(y_train, clf_pc.predict(pc_treino))
accuracy_test_pc = accuracy_score(y_test, y_pred_pc)
print(f"Acurácia do modelo Decision Tree com PCA (treinamento): {accuracy_train_pc:.4f}")
print(f"Acurácia do modelo Decision Tree com PCA (teste): {accuracy_test_pc:.4f}")

Acurácia do modelo Decision Tree com PCA (treinamento): 0.8927
Acurácia do modelo Decision Tree com PCA (teste): 0.8242
CPU times: total: 547 ms
Wall time: 201 ms


A acurácia com PCA e padronização foi menor do que a acurácia com PCA sem padronização pois a padronização removeu a influência das variáveis com maior variabilidade, que poderiam ser mais relevantes ou menos relevantes para a classificação.